In [1]:
import boto3
import os
from dotenv import load_dotenv

In [2]:
# Load the environment variables from the .env file
load_dotenv()

True

In [18]:
# Creating the DynamoDB Client
# make sure to set the AWS_ACCESS_KEY_ID and
# AWS_SECRET_ACCESS_KEY environment variables
# OR pass credentials when creating the client:
dynamodb_client = boto3.client('dynamodb', region_name="us-east-1")

TABLE_NAME = "fumehoods"

dyn_resource = boto3.resource("dynamodb", region_name="us-east-1")
TABLE_NAME = "fumehoods"
table = dyn_resource.Table(TABLE_NAME)

hoods = {
    "id": "hoods",
    "map": {
        "Biotech.Floor_3.Lab_317.Hood_1": {
            "building": "Biotech",
            "floor": "3",
            "lab": "317",
            "hood": "1",
            "flow_sensor": "#path_to_this_sensor",
        },
        "Biotech.Floor_4.Lab_433.Hood_1": {
            "building": "Biotech",
            "floor": "4",
            "lab": "433",
            "hood": "1",
            "flow_sensor": "#path_to_this_sensor",
        },
    },
}

buildings = {
    "id": "buildings",
    "map": {
        "Biotech": {"external_temp_sensor": "#path_to..."},
        "Weill": {"external_temp_sensor": "#path_to..."},
    },
}

# Put the above data into the table
with table.batch_writer() as writer:
    response = writer.put_item(Item=hoods)
    print(response)
    response = writer.put_item(Item=buildings)
    print(response)

# # Put the above data into the table
# response = dynamodb_client.put_item(TableName=TABLE_NAME, Item=hoods)
# print("put_item response:")
# display(response)

# Print what was just added
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}
)
print("get_item response:")
display(response["Item"])

None
None
get_item response:


{'id': {'S': 'hoods'},
 'map': {'M': {'Biotech.Floor_3.Lab_317.Hood_1': {'M': {'flow_sensor': {'S': '#path_to_this_sensor'},
     'floor': {'S': '3'},
     'lab': {'S': '317'},
     'hood': {'S': '1'},
     'building': {'S': 'Biotech'}}},
   'Biotech.Floor_4.Lab_433.Hood_1': {'M': {'flow_sensor': {'S': '#path_to_this_sensor'},
     'floor': {'S': '4'},
     'lab': {'S': '433'},
     'hood': {'S': '1'},
     'building': {'S': 'Biotech'}}}}}}

ClientError: An error occurred (ValidationException) when calling the GetItem operation: The provided key element does not match the schema

In [4]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

print(table.key_schema)

[{'AttributeName': 'id', 'KeyType': 'HASH'}]


In [5]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

response = table.scan(ProjectionExpression="id")
data = response['Items']

# Uncomment for large queries
# while 'LastEvaluatedKey' in response:
#     response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
#     data.extend(response['Items'])

data

[{'id': 'Biotech.Floor_4.Lab_433.Hood_1'},
 {'id': 'BiotechnologyBuilding/lab_433/hood'},
 {'id': 'buildings'},
 {'id': 'cat'},
 {'id': 'Biotech.Floor_3.Lab_317.Hood_1'},
 {'id': 'maggie'},
 {'id': 'Biotech.Floor_4.Lab_433.Hood_2'},
 {'id': 'hoods'},
 {'id': 'Biotech.Floor_4.Lab_441.Hood_1'}]